# Optimization Gone Wrong and How to Fix it!

In [ ]:
%pip -q install gurobipy==13.0.0

In [ ]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

## Getting to Know a Parameter with Gurobot


Here is a variation of the `quadratic assignment problem (QAP)`. Below is a formulation. We really won't need it but it's here for fun.

#### QAP Mathematical Formulation

##### Sets and Indices
- $i, k \in \{0, 1, \ldots, n-1\}$: Facilities
- $j, l \in \{0, 1, \ldots, n-1\}$: Locations

##### Parameters
- $f_{ik}$: Flow between facility $i$ and facility $k$ (represented as `flow[i][k]`)
- $d_{jl}$: Distance between location $j$ and location $l$ (represented as `dist[j][l]`)
- $n$: Number of facilities (and locations)

##### Decision Variables
- $x_{ij} \in \{0, 1\}$: Binary variable equal to 1 if facility $i$ is assigned to location $j$, 0 otherwise

##### Objective Function

$$
\min \sum_{i=0}^{n-1} \sum_{j=0}^{n-1} \sum_{k=0}^{n-1} \sum_{l=0}^{n-1} c_{ik} \cdot f_{ik} \cdot d_{jl} \cdot x_{ij} \cdot x_{kl}
$$

where the coefficient $c_{ik}$ is defined as:

$$
c_{ik} = \begin{cases} 
-1 & \text{if } (i+k) \bmod 2 = 0 \\
1 & \text{if } (i+k) \bmod 2 = 1
\end{cases}
$$

##### Constraints

**Assignment constraints - Each facility to exactly one location:**
$$
\sum_{j=0}^{n-1} x_{ij} = 1 \quad \forall i \in \{0, 1, \ldots, n-1\}
$$

**Assignment constraints - Each location receives exactly one facility:**
$$
\sum_{i=0}^{n-1} x_{ij} = 1 \quad \forall j \in \{0, 1, \ldots, n-1\}
$$

**Binary constraints:**
$$
x_{ij} \in \{0, 1\} \quad \forall i, j \in \{0, 1, \ldots, n-1\}
$$

Use this data and the model below to solve the problem. You may notice it take a little while. 
- If you are running this on your laptop, set $n = 10$ and it should take 20-30 seconds.
- If you are running in Colab, set $n = 9$. If it's set to 10, it could take around 3 or 4 minutes. 

In [ ]:
n = 10

# Distance between locations
dist = [[abs(j - k) + 1 for k in range(n)] for j in range(n)]

# Flow/weight between facilities
flow = [[(i + k + 1) * ((i - k) ** 2 + 1) for k in range(n)] for i in range(n)]

Here is the model in `gurobipy`.

In [ ]:
# Quadratic assignment problem
m1 = gp.Model()
m1.Params.Seed = 42

# x[i,j] = 1 if facility i is assigned to location j
x = m1.addVars(n, n, vtype=GRB.BINARY, name="x")

# Build objective function
obj = gp.QuadExpr()
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                coeff = flow[i][k] * dist[j][l]
                # Alternate signs
                if (i + k) % 2 == 0:
                    coeff = -coeff
                obj.addTerms(coeff, x[i, j], x[k, l])

m1.setObjective(obj, GRB.MINIMIZE)

# Each facility is assigned to exactly one location
for i in range(n):
    m1.addConstr(x.sum(i, "*") == 1, name=f"assign_facility[{i}]")

# Each location is assigned exactly one facility
for j in range(n):
    m1.addConstr(x.sum("*", j) == 1, name=f"assign_to_location[{j}]")

Run this without editing any parameters to see how long it takes. 

In [ ]:
# Set parameters here after running in once. 

# m.Params.ParameterName = value
m1.optimize()

Let's try to speed this up. You can...
- Try to recall a parameter mentioned in the log file session that may fit this type of problem and searching the [documentation](https://docs.gurobi.com/projects/optimizer/en/current/). This is a good idea. 

OR

- Ask [Gurobot](https://portal.gurobi.com/iam/chat/). Even try giving it the log file just produced to see what it suggests. This is also a good idea. 

<div style="
  border-left: 6px solid #f59e0b;  /* amber/yellow */
  padding:14px 16px;
  border-radius:10px;
  margin:12px 0;
  font-size:16px; line-height:1.4;">
  <strong style="font-size:18px;">🧠 Take a minute to think about this one!</strong><br>
  <em>Pause here.</em> There are spoilers just below. 
</div>

### Click to expand and see the answer

What we are looking for is `PreQLinearize`. Ask [Gurobot](https://portal.gurobi.com/iam/chat/) for an explanation on what this does. To see significant improvement, set `PreQLinearize = 2` in the above a commented out line of code and see what happens. 

## Handling Infeasibility with IIS and feasRelax

Of course we go back to the good ol' widgets transportation problem. Here is a quick reminder of this now classic (infamous?) problem.

We make widgets. Have a set of production facilities that produce boxes of widgets. There is also a set of distribution locations that will then distribute the widgets for sale. Each distribution center has a forecasted demand and each production facility has a min and max number of widgets it can make during this period.

But times have changed since we first optimized our original transportation problem. Management has some more strict requirements for production while meeting demand. Each facility must produce at least 90% of its max production. AND the min amount of widgets to be shipped from any production to distribution facility (if it's nonzero) must be at least 50. 

In [ ]:
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'
transp_cost = pd.read_csv(path + 'cost.csv')
# get production and distribution locations from data frame
production = list(transp_cost['production'].unique())
distribution = list(transp_cost['distribution'].unique())
transp_cost = transp_cost.set_index(['production','distribution']).squeeze()

max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand")

# the min production is a fraction of the max
frac = 0.90
# min shipment size, if it's > 0
C = 50

### The Original Model

- Our **decision variables** are the amount produced at facility $p$ and shipped to distribution center $d$, denoted $x_{p,d}$
- We have **constraints** to ensure:
  - Min and max production
  - Demand is met
- The **objective** is to meet demand at **minimal cost**.

Our production and distribution `sets` are:
$$
\begin{align*}
P &= \{\texttt{'Baltimore', 'Cleveland', 'Little Rock', 'Birmingham', 'Charleston'}\}, &\texttt{production} \\
D &= \{\texttt{'Columbia', 'Indianapolis', 'Lexington', 'Nashville', 'Richmond', 'St. Louis'}\}  &\texttt{distribution}
\end{align*}
$$

Model `parameters`:
$$
\begin{align*}
c_{p,d} &: \text{cost to ship a widget from} \space p \space \text{to} \space d &\texttt{trasp}\_\texttt{cost[p,d]}\\
m_p &: \text{maximum a production facility} \space p \space\text{can produce} &\texttt{max}\_\texttt{prod[p]}\\
n_d &: \text{demand at distribution hub} \space d &\texttt{n}\_\texttt{demand[d]}
\end{align*}
$$


As a reminder, here is the original *formulation*:
$$
\begin{align*}
{\rm min} &\sum_{p,d}c_{p,d}x_{p,d}\\
{\rm s.t.}\\
&\sum_{d}x_{p,d} \le m_p, &\forall p \in P \quad &\texttt{can}\_\texttt{produce[p]}\\
&\sum_{d}x_{p,d} \ge a*m_p,&\forall p \in P \quad &\texttt{must}\_\texttt{produce[p]}\\
&\sum_{p}x_{p,d} \ge n_d, &\forall d \in D \quad &\texttt{meet}\_\texttt{demand[d]}\\
&x_{p,d} \ge 0,  &\forall p \in P, d \in D\quad &\texttt{non-negativity}\\
\end{align*}
$$

Here is our original model. Run this scenario as described. I wonder what happens...

In [ ]:
# gurobipy code for this above formulation
m = gp.Model('widgets')

# decision vars
x = m.addVars(production, distribution, vtype=GRB.SEMICONT, lb = C, name = 'prod_ship')

# constraints
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = 'can_produce')
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = 'must_produce')
meet_demand = m.addConstrs((x.sum('*', d) == n_demand[d] for d in distribution), name = "meet_demand")

#objective
m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)
m.optimize()

<div style="
  border-left: 6px solid #f59e0b;  /* amber/yellow */
  padding:14px 16px;
  border-radius:10px;
  margin:12px 0;
  font-size:16px; line-height:1.4;">
  <strong style="font-size:18px;">💬 Explainability Moment</strong><br>
  <em>This model is infeasible!</em> How would you explain what infeasibility means to a <b>non-technical stakeholder</b> in this context? 
</div>


### Computing IIS

If you don't know what an IIS is, try and ask [Gurobot](https://portal.gurobi.com/iam/chat)? 

If you'd rather not do that right now, here's a quick definition.

An **IIS (Irreducible Infeasible Subsystem)** is a minimal subset of constraints and variable bounds in an infeasible optimization model that satisfies two key properties:

- It is itself infeasible - the subset alone makes the model infeasible.
- It becomes feasible if any single constraint or bound is removed - removing any one element from the subset results in a feasible subsystem.

And here is how to compute one:

In [ ]:
if m.Status == GRB.INFEASIBLE:
    m.computeIIS()
    
    # Check which constraints are in the IIS
    for constr in m.getConstrs():
        if constr.IISConstr:
            print(f"Conflicting: {constr.ConstrName}")

<div style="
  border-left: 6px solid #f59e0b;  /* amber/yellow */
  padding:14px 16px;
  border-radius:10px;
  margin:12px 0;
  font-size:16px; line-height:1.4;">
  <strong style="font-size:18px;">💬 Explainability Moment</strong><br>
  Now that we see where the issue is, how would you go about fixing this? There are a bunch of ways to do it, so there's no single correct answer. How would you explain each to stakeholders?
</div>

Here is the data and model again. Make it feasible! 

In [ ]:
max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand")

# the min production is a fraction of the max
frac = 0.90
# min shipment size, if it's > 0
C = 50

# gurobipy code for this above formulation
m = gp.Model('widgets')

# decision vars
x = m.addVars(production, distribution, vtype=GRB.SEMICONT, lb = C, name = 'prod_ship')

# constraints
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = 'can_produce')
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = 'must_produce')
meet_demand = m.addConstrs((x.sum('*', d) == n_demand[d] for d in distribution), name = "meet_demand")

#objective
m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)
m.optimize()

### Using feasRelax
IIS computes the subset of constraints in which we know where the problem lies. Another way would be to use `feasRelax`, which relaxes certain parts of the model to make sure it's feasible. In this case we are using `feasRelaxS` which is a little easier to use right out of the box as it relaxes all constraints equally. 

Below is some code to run a relaxed model, along with what each argument does in the function used. Check out the documentation on [feasRelaxS](https://docs.gurobi.com/projects/optimizer/en/current/reference/python/model.html#Model.feasRelaxS). 

In [ ]:
# Relax ALL constraints equally (penalty = 1.0 for each)
m.feasRelaxS(0, False, False, True)
#            │    │      │      └─ crelax: True = relax constraints
#            │    │      └─ vrelax: False = don't relax variable bounds
#            │    └─ minrelax: False = just minimize violations
#            └─ relaxobjtype: 0=sum, 1=sum of squares, 2=count

m.optimize()

The model is now minimizing the sum of the total violations. To help us with that, let's first write an `lp file` to see what exactly is happening. 

In [ ]:
m.write("relax.lp")

The relaxed model adds on a positive and negative violation artificial variable for each constraint. Looking at the lp file, how can we select the artificial variables?

In [ ]:
# Analyze violations
print("\nViolated constraints:")
for var in m.getVars():
    if var.VarName.startswith(  ????  ) and var.X > 1e-6:
        print(f"  {var.VarName}: {var.X:.4f}")

So there is a problem with the production amount in a particular production location. Specifically, the minimum amount we are *requiring* to be produced. Go back to the lp file to investigate further. 

Let $A_{\hat{p}}$ be the artificial variable noted above for production location $\hat{p}$. The constraint in question, looking at the lp file, is:

$$
\begin{align*} 
\sum_{d}x_{\hat{p},d} + A_{\hat{p}} &\ge 126
\end{align*}
$$

<div style="
  border-left: 6px solid #f59e0b;  /* amber/yellow */
  padding:14px 16px;
  border-radius:10px;
  margin:12px 0;
  font-size:16px; line-height:1.4;">
  <strong style="font-size:18px;">💬 Explainability Moment</strong><br>
Now that you see what this looks like algebraically, how would you communicate this?
</div>

<div style="
  border-left: 6px solid #3b82f6;  /* blue */
  padding:14px 16px;
  border-radius:10px;
  margin:12px 0;
  font-size:16px; line-height:1.4;">
  <strong style="font-size:18px;">📘 Optional Activity: Try different argument values for feasRealaxS</strong><br>
  For example, we minimized the sum of total constraint violations. Check the other options and see how that changes things. 
</div>

#### Homework problem!

<div style="
  border-left: 6px solid #3b82f6;  /* blue */
  padding:14px 16px;
  border-radius:10px;
  margin:12px 0;
  font-size:16px; line-height:1.4;">
  <strong style="font-size:18px;">📘 Optional Activity: Use your new skills!</strong><br>
  The below model is infeasible. Fix it however you see fit!
</div>


In [ ]:
# gurobipy code for this above formulation
m2 = gp.Model('widgets')

max_prod2 = pd.Series([210,225,140,130,220], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand")

frac = 0.75
# decision vars
x2 = m2.addVars(production, distribution, vtype=GRB.SEMICONT, lb = C, name = 'prod_ship')
y2 = m2.addVars(production, vtype=GRB.BINARY, name = 'prod_on')

# constraints
can_produce = m2.addConstrs((gp.quicksum(x2[p,d] for d in distribution) <= max_prod2[p] for p in production), name = 'can_produce')
must_produce = m2.addConstrs((gp.quicksum(x2[p,d] for d in distribution) >= frac*max_prod2[p] for p in production), name = 'must_produce')
meet_demand = m2.addConstrs((x2.sum('*', d) == n_demand[d] for d in distribution), name = "meet_demand")
xy_link = m2.addConstrs((x2[p,d] <= max_prod2[p]*y2[p] for p in production for d in distribution), name = 'xy_link')
only_four = m2.addConstr((y2.sum() <= 4), name = 'only4')

#objective
m2.setObjective(gp.quicksum(transp_cost[p,d]*x2[p,d] for p in production for d in distribution), GRB.MINIMIZE)
m2.optimize()